### Двухуровневая рекомендательная система.

##### Входные данные:
 - data - данные по продажам
 - item_features - данные по товарам
 - user_features - данные по покупателям
 - test_data - тестовые данные по продажам для финального тестирования модели
 
##### Задача: построить рекомендательную систему по товарам.  
##### Целевая метрика - money precision @ 5. Целевое значение - money precision @ 5 > 20%  

##### Бизнес ограничения в топ-5 товарах:  
- Для каждого юзера 5 рекомендаций
- 2 новых товара (юзер никогда не покупал)
- 1 дорогой товар, > 7 долларов
- Все товары из разных категорий (категория - sub_commodity_desc)
- Стоимость каждого рекомендованного товара > 1 доллара

##### Выходной формат данных - .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)  

Реализуем двухуровневую рекомендательную систему по схеме Implicit.ALS + LightGBM


----

##### Реализация  (пайплайн)  
- загружаем данные
- разбиваем на трейн/тесты в соответствии с 2 уровнями
- осуществляем предфильрацию
- обучаем рекоммендер первого уровня. при обучении используем tfidf-взвешивание, берем own_rec - прочие были отметены опытным путем

- готовим фичи для товаров: 
 * эмбеддинги
 * цена
 * среднее кол-во товара в корзине
 * накопительная выручка по товару
 * кол-во товаров в той же категории
 * кол-во дней с последней продажи. если продаж за период не было, то берем кол-во дней в периоде и умножаем на 2 (типа вес)
 * оставшиеся фичи преобразуем в категориальные  

- готовим фичи для юзеров:
 * эмбеддинги
 * средний чек
 * дней с последней покупки. если покупок за период не было, то берем кол-во дней в периоде и умножаем на 2 (типа вес)
 * преобразуем возраст, средний доход, размер дома и кол-во детей в числовой формат
 * оставшиеся фичи преобразуем в категориальные

- обучаем модель второго уровня. в качестве результата берем скор предикта.
- по скорам отбираем для юзера рекомендованные товары (100)
- по бизнес-требованиям из них отбираем по 5 товаров
- считаем метрику

- с помощью обученной модели считаем предикт для тестовых данных, считаем метрику.

#### Что можно было бы еще:  
 - попробовать на первом уровне вместо бейзлайна использовать гибридную модель. первые попытки ощутимого результата не дали, поэтому было отложено.
 - попробовать дополнительные фичи как для товаров и юзеров, так и для пар юзер-товар. 
 - попробовать gridsearch
 - Попробовать иные лоссы
 - пред-фильтровать данные для модели 2 уровня
 - попробовать на втором уровне XGBoost или нейронку.  
 - выполнить более качественный рефакторинг - добавить функциональную обработку данных или вынести второй уровень в отдельный класс с написанием соответствующих методов
   
это все был отложено, так как полуенный результат в принципе пока устроил, а времени катастрофически не хватало.

In [3]:
!pip install implicit lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/SARAWAN/recomend_system/

/content/drive/MyDrive/Colab Notebooks/SARAWAN/recomend_system


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys  #нужно для определения расположения файла (главной директории)
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k, money_precision_at_k
from src.utils import pre_filter_items, get_users_features, get_items_features, get_recommendation_5
from src.recommenders import MainRecommender
from src.recommenders import ColdRecommender

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

Загружаем данные

In [7]:
path = '/content/drive/MyDrive/Colab Notebooks/SARAWAN/recomend_system/data/'
path_data = path + 'retail_train.csv'  # ниже загружаю уже с расчитанной ценой
path_features = path + 'product.csv'
path_user = path + 'hh_demographic.csv'

data = pd.read_csv(path_data)
item_features = pd.read_csv(path_features)
user_features = pd.read_csv(path_user)

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [ ]:
# data.shape #test_data.shape

In [8]:
#сразу считаем цены - они нам потребуются далее
prices = data.groupby(['item_id'])['sales_value'].mean().reset_index()
sales_qty = data.groupby(['item_id'])['quantity'].mean().reset_index()
prices = prices.merge(sales_qty, on='item_id', how='left')
prices['price'] = [prices.iloc[i]['sales_value'] / prices.iloc[i]['quantity']\
                   if prices.iloc[i]['quantity'] > 0 else 0 for i in prices['item_id'].index]
prices.drop(columns=['sales_value', 'quantity'], axis=1, inplace=True)

In [9]:
data = data.merge(prices, on='item_id', how='left')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


Расчет цены делается не особо быстро - оставил возможность загруки готовых данных на будущее.

In [ ]:
# path_data_with_prices = path + 'data_prices.csv'
# data.to_csv(path_data_with_prices, index=False)

In [10]:
# data = pd.read_csv(path_data_with_prices)
data.head(2)
# если что - не забыть дропнуть цены - вынести их в отдельный датасет!

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


In [11]:
# Схема разбиения: все данные -> 6 недель -> 3 недели
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] <= data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  
data_val_lvl_2 = data[data['week_no'] > data['week_no'].max() - val_lvl_2_size_weeks]

In [12]:
# Вывод сведений о данных
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data['user_id'].nunique()} Items: {df_data['item_id'].nunique()}")

In [13]:
# ищем общих пользователей
common_users = list(set(data_train_lvl_1.user_id.values)&(set(data_train_lvl_2.user_id.values))&set(data_val_lvl_2 .user_id.values))
out_lvl_2 = list(set(data_train_lvl_1.user_id.values) - set(data_train_lvl_2.user_id.values))
out_val_2 = list(set(data_train_lvl_2.user_id.values) - set(data_val_lvl_2.user_id.values))
out_users_all = (out_lvl_2 + out_val_2)
print('На момент обучения модели было: {} пользователей'.format(data_train_lvl_1.user_id.nunique()))
print('Постоянные "горячие" пользователи: {}'.format(len(common_users)))
print('Не вошедшие на 2 уровень пользователи:', len(out_lvl_2)) 
print('Не вошедшие на 2 уровень на тест пользователи:', len(set(out_val_2)))
print('Всего:', len(set(out_users_all)))

data_train_lvl_1 = data_train_lvl_1[data_train_lvl_1.user_id.isin(common_users)]
data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1.user_id.isin(common_users)]
data_train_lvl_2 = data_train_lvl_2[data_train_lvl_2.user_id.isin(common_users)]
data_val_lvl_2 = data_val_lvl_2[data_val_lvl_2.user_id.isin(common_users)]

print_stats_data(data_train_lvl_1,'train_1')
print_stats_data(data_val_lvl_1,'val_1')
print_stats_data(data_train_lvl_2,'train_2')
print_stats_data(data_val_lvl_2,'val_2')

На момент обучения модели было: 2498 пользователей
Постоянные "горячие" пользователи: 1815
Не вошедшие на 2 уровень пользователи: 302
Не вошедшие на 2 уровень на тест пользователи: 381
Всего: 683
train_1
Shape: (1860128, 13) Users: 1815 Items: 77286
val_1
Shape: (190186, 13) Users: 1815 Items: 29016
train_2
Shape: (190186, 13) Users: 1815 Items: 29016
val_2
Shape: (83765, 13) Users: 1815 Items: 20380


In [14]:
# Тренировочные данные для 1 уровня
data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


# Очистим выборку от непопулярных товаров

In [15]:
# Предфильтрация
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1, out_users = pre_filter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 77286 to 5001


In [16]:
items_sold = data_train_lvl_1['item_id'].unique().tolist()
print(f'Отобрано категорий - {item_features[item_features["item_id"].isin(items_sold)]["sub_commodity_desc"].nunique()}')

Отобрано категорий - 724


# Обучим MainRecommender

In [18]:
# обучаем рекоммендер первого уровня
recommender = MainRecommender((data_train_lvl_1), weighting=False)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [19]:
# Сохраняем эмбеддинги - будут использованы далее в качестве фичей и словари перевода
items_embeddings = recommender.items_emb_df
users_embeddings = recommender.users_emb_df
id_to_itemid = recommender.id_to_itemid
itemid_to_id = recommender.itemid_to_id
user_item_matrix_pivot = recommender.user_item_matrix_pivot
overall_top_purchases = recommender.overall_top_purchases


Строим рекомендации первого уровня

In [20]:
# # actual - фактически купленные товары
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96..."


## N = 200 - Кол-во рекомендаций для юзера

In [21]:
N = 200 # Кол-во рекомендаций для юзера

users = set(data_train_lvl_1['user_id'].unique().tolist())
top_popular_items = recommender.overall_top_purchases[:N]

In [22]:
# Строим рекомендации
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].progress_apply(lambda x: \
                                                                             recommender.get_own_recommendations(x, N=N))

100%|██████████| 1815/1815 [00:01<00:00, 1167.77it/s]


In [23]:
# Строим рекомендации
result_lvl_1['als_recommendations'] = result_lvl_1['user_id'].progress_apply(lambda x: \
                                                                             recommender.get_als_recommendations(x, N=N))

100%|██████████| 1815/1815 [00:03<00:00, 533.35it/s]


In [24]:
result_lvl_1['top_pop'] = result_lvl_1['user_id'].progress_apply(lambda x: \
                                                                             recommender.overall_top_purchases[:N])

100%|██████████| 1815/1815 [00:00<00:00, 70494.14it/s]


In [25]:
result_lvl_1.tail(2)

,user_id,actual,own_recommendations,als_recommendations,top_pop
1813,2498,"[865511, 962991, 1076374, 1102358, 5564901, 15...","[1022066, 9885206, 5995483, 9526100, 1070719, ...","[1053690, 862349, 1082185, 9526410, 1070820, 9...","[1082185, 1029743, 995242, 1106523, 981760, 11..."
1814,2500,"[856455, 902192, 903476, 931672, 936634, 95170...","[12263035, 9859060, 12263279, 5577022, 1053628...","[1029743, 995242, 1058997, 859075, 833025, 108...","[1082185, 1029743, 995242, 1106523, 981760, 11..."


In [26]:
result_lvl_1.progress_apply(lambda row: precision_at_k(row['own_recommendations'], row['actual']), axis=1).mean() 

100%|██████████| 1815/1815 [00:00<00:00, 6001.63it/s]


0.0765840220385675

In [27]:
result_lvl_1.progress_apply(lambda row: precision_at_k(row['als_recommendations'], row['actual']), axis=1).mean() 

100%|██████████| 1815/1815 [00:00<00:00, 4961.15it/s]


0.25112947658402207

In [28]:
result_lvl_1.progress_apply(lambda row: precision_at_k(row['top_pop'], row['actual']), axis=1).mean() 

100%|██████████| 1815/1815 [00:00<00:00, 4324.29it/s]


0.2678787878787879

# Рекомендации второго уровня

In [29]:
# Отберем уникальных юзеров для обучения 2 уровня
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']
# Пока только warm start - для новых используем бейзлайн
train_users = data_train_lvl_1['user_id'].unique()

# Не попавшие на 2-й уровень
# юзеры, которые не попали в выборку , то есть новые, для них используем Сold_recomendation или другие способы рекомендаций
out_lvl_2 = list(set(set(data_train_lvl_1.user_id.values) - set(users_lvl_2.user_id.values)))
print('out_lvl_2: ', len(out_lvl_2))
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]



out_lvl_2:  0


In [30]:
# добавляем рекомендации с первого уровня
users_lvl_2 = users_lvl_2.merge(result_lvl_1, on='user_id', how='left')
users_lvl_2.tail(2)

,user_id,actual,own_recommendations,als_recommendations,top_pop
1813,1615,"[822238, 908940, 916381, 1092118, 1117500]","[14043825, 14050436, 6773229, 10182813, 983561...","[995242, 1029743, 1082185, 844165, 981760, 105...","[1082185, 1029743, 995242, 1106523, 981760, 11..."
1814,935,[6534178],"[1012052, 9526628, 1044153, 1068957, 1064574, ...","[1082185, 981760, 1127831, 1024306, 860776, 98...","[1082185, 1029743, 995242, 1106523, 981760, 11..."


In [ ]:
users_lvl_2.shape

(1815, 5)

# Генерируем фичи для второго уровня.  
## Юзеры.

In [31]:
user_features_upd = get_users_features(user_features, data_train_lvl_2, users_embeddings)[0] # внешних фичей всего 801, а пользователей 2500
user_features_upd.head(2)

,user_id,marital_status_code,homeowner_desc,hh_comp_desc,age_desc_int,kid_category_desc_int,days_from_last_purchase,av_check,av_items_num,0,...,90,91,92,93,94,95,96,97,98,99
0,1,A,Homeowner,2 Adults No Kids,70,0,3,51.265000,23.375000,1.365743,...,0.466952,-4.377888,-6.646237,0.403586,-0.991355,5.011954,6.526209,-3.686384,5.367149,1.364787
1,7,A,Homeowner,2 Adults No Kids,50,0,4,41.318571,23.714286,5.394765,...,-5.465224,0.387922,-1.402136,2.551402,-5.576015,0.392229,6.243781,1.340934,2.435713,3.860173


In [32]:
user_features_upd.shape

(801, 109)

 Товары

In [33]:
# надо перезапустить ячейку 3
item_features_upd = get_items_features(item_features, data_train_lvl_2, items_embeddings)[0]

In [34]:
#Добавим цены - надо бы внести в функцию подготовки фичей по товарам!
item_features_upd = item_features_upd.merge(prices, on='item_id', how='left')

Готовим данные для обучения второго уровня.  

In [35]:
# # Создаем колонку товаров, которые покупал сам пользователь
s = users_lvl_2.apply(lambda x: pd.Series(x['als_recommendations']), axis=1).stack().reset_index(level=1 , drop=True)
s.name = 'item_id'
users_lvl_2 = users_lvl_2.drop('als_recommendations', axis=1).join(s)
users_lvl_2 = users_lvl_2.drop('own_recommendations', axis=1)
users_lvl_2 = users_lvl_2.drop('actual', axis=1)
users_lvl_2['drop'] = 1  # фиктивная переменная
users_lvl_2 = users_lvl_2.drop('top_pop', axis=1)

In [36]:
users_lvl_2

,user_id,item_id,drop
0,2070,1096036,1
0,2070,983584,1
0,2070,1006184,1
0,2070,1029743,1
0,2070,1038217,1
...,...,...,...
1814,935,921744,1
1814,935,1119051,1
1814,935,6533936,1
1814,935,882288,1


In [37]:
data_train_lvl_2.shape # Проверить фильтрацию надо

(190186, 13)

In [38]:
targets_lvl_2 = data_train_lvl_2[['user_id','item_id']].copy()
targets_lvl_2['target'] = 1  # тут покупки 
targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
targets_lvl_2['target'].fillna(0, inplace= True) # тут не было покупок
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [39]:
targets_lvl_2.head(2)

,user_id,item_id,target
0,2070,1096036,1.0
1,2070,983584,0.0


In [40]:
targets_lvl_2 = targets_lvl_2.merge(user_features_upd, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(item_features_upd, on='item_id', how='left')

In [41]:
targets_lvl_2.shape

(382275, 222)

In [42]:
targets_lvl_2.head(2)

,user_id,item_id,target,marital_status_code,homeowner_desc,hh_comp_desc,age_desc_int,kid_category_desc_int,days_from_last_purchase,av_check,...,91_y,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y,price
0,2070,1096036,1.0,U,Unknown,Unknown,50.0,0.0,0.0,17.370727,...,0.013431,-0.008438,0.010454,-0.003419,-0.019758,-0.000778,0.005914,-0.004299,-0.014483,1.442297
1,2070,983584,0.0,U,Unknown,Unknown,50.0,0.0,0.0,17.370727,...,-0.001919,-0.000840,0.003572,0.019623,-0.009723,-0.003606,-0.014814,0.011094,0.011071,2.288489


Обучаем 2 уровень. Предфильтрацию здесь пока не делаю!

In [43]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [ ]:
#X_train.info()

In [44]:
cat_feats = ['marital_status_code', 'homeowner_desc', 'hh_comp_desc', 'manufacturer', 'department', 'brand', 
             'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']

In [45]:
#%%time
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats, boosting_type='goss', random_state=42)
lgb.fit(X_train, y_train)

LGBMClassifier(categorical_column=['marital_status_code', 'homeowner_desc',
                                   'hh_comp_desc', 'manufacturer', 'department',
                                   'brand', 'commodity_desc',
                                   'sub_commodity_desc',
                                   'curr_size_of_product'],
               max_depth=7, objective='binary', random_state=42)

In [ ]:
# ? lgb

Строим предикт для 2 уровня.

In [46]:
targets_lvl_3 = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_3.drop_duplicates(keep='first', inplace=True)

In [47]:
targets_lvl_3 = targets_lvl_3.merge(user_features_upd, on='user_id', how='left')
targets_lvl_3 = targets_lvl_3.merge(item_features_upd, on='item_id', how='left')
targets_lvl_3.shape

(75950, 221)

In [48]:
preds = lgb.predict(targets_lvl_3)
test_preds_proba = lgb.predict_proba(targets_lvl_3) #[:, 1]

In [49]:
targets_lvl_3['res'] = test_preds_proba[:, 1]
targets_lvl_3.head(2)

,user_id,item_id,marital_status_code,homeowner_desc,hh_comp_desc,age_desc_int,kid_category_desc_int,days_from_last_purchase,av_check,av_items_num,...,92_y,93_y,94_y,95_y,96_y,97_y,98_y,99_y,price,res
0,2449,721164,A,Homeowner,2 Adults Kids,40.0,3.0,4.0,29.9992,12.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.99000,0.002173
1,314,820486,U,Homeowner,2 Adults Kids,50.0,1.0,27.0,157.3050,84.50,...,0.003614,0.002032,0.006738,0.010882,0.006356,0.007755,0.002219,-0.000543,2.09358,0.383176


In [50]:
targets_lvl_3['user_id'].nunique()

1815

In [51]:
targets_lvl_3.sort_values(['user_id', 'res'], ascending=False, inplace=True)
recs = targets_lvl_3.groupby('user_id')['item_id']
recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

In [52]:
recomendations = pd.DataFrame(recomendations)

result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(recomendations, on='user_id', how='left')
result_lvl_2.head(2)

,user_id,actual,recomendations
0,1,"[883616, 917704, 931860, 961554, 995242, 10020...","[1082185, 995242, 8091601, 1074612, 9297615, 1..."
1,6,"[909479, 7431990, 6553035, 12263667, 13382461,...","[995242, 7431990, 849843, 1015296, 887325, 840..."


Отбираем дорогие товары (> $7)

In [53]:
top_valued_items = prices[(prices['price'] > 7)]
top_valued_items = top_valued_items.sort_values(by='price', ascending=False, na_position='last')
top_valued_items_list = top_valued_items['item_id'].tolist()

In [54]:
# отфильтруем < $1
top_popular_items_m = prices[(prices['price'] > 1) & (prices['item_id'].isin(top_popular_items))]['item_id'].tolist() 
print(f'Популярных товаров до фильтрации - {len(top_popular_items)}, после - {len(top_popular_items_m)}')

Популярных товаров до фильтрации - 200, после - 150


In [55]:
top_valued_items_list = top_valued_items['item_id'].tolist()

In [56]:
result_lvl_2['rec'] = result_lvl_2['user_id'].progress_apply\
        (lambda x: get_recommendation_5(x, result_lvl_2, item_features, top_popular_items_m, top_valued_items_list)[0])

100%|██████████| 1815/1815 [01:18<00:00, 23.16it/s]


In [57]:
result_lvl_2.head(2)

,user_id,actual,recomendations,rec
0,1,"[883616, 917704, 931860, 961554, 995242, 10020...","[1082185, 995242, 8091601, 1074612, 9297615, 1...","[12757425, 824005, 832678, 1082185, 995242]"
1,6,"[909479, 7431990, 6553035, 12263667, 13382461,...","[995242, 7431990, 849843, 1015296, 887325, 840...","[1329768, 824005, 832678, 995242, 7431990]"


Считаем целевую метрику на сформированных рекомендациях

In [58]:
result_lvl_2.progress_apply(lambda row: precision_at_k(row['rec'], row['actual'], k=5), axis=1).mean() 

100%|██████████| 1815/1815 [00:00<00:00, 16775.66it/s]


0.5101928374655648

In [59]:
result_lvl_2.progress_apply(lambda row: money_precision_at_k(row['rec'], row['actual'], prices), axis=1).mean()

100%|██████████| 1815/1815 [00:04<00:00, 366.14it/s]


0.5378593503236332

# "холодные " рекомендации новым пользователям

In [84]:
# Отбираем пользователей, у которых заполены признаки
uid = user_features['user_id'].tolist()

# Временной интервал - 3 недели???
test_size_weeks = 6

# Тренировочные данные
data_train_cold = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test_cold = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train_cold = data_train_cold[data_train_cold.user_id.isin(uid)]

# Предфильтрация
n_items_before = data_train_cold['item_id'].nunique()
data_train_cold = pre_filter_items(data_train_cold, item_features=item_features, take_n_popular=5000)[0]
data_train_cold = data_train_cold[data_train_cold.item_id != 999999]
n_items_after = data_train_cold['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

# Отбираем товары из топ 5000
items_sold = data_train_cold['item_id'].unique().tolist()
items_features_sold = item_features[item_features["item_id"].isin(items_sold)]
print(items_features_sold.shape)
print(data_train_cold.user_id.nunique())

# Пользователи для тренировки 
# Cоздаем фичи для пользователей и товаров , users_embeddings, items_embeddings - это получаем из класса MainRecommender (надо бы вбить в функцию значение по умолчанию - 0)
user_features_ext = get_users_features(user_features, data_train_cold, users_embeddings)[1]
item_features_ext = get_items_features(items_features_sold, data_train_cold, items_embeddings)[1]

# Пользователи для теста
data_test_cold = pre_filter_items(data_test_cold, item_features=item_features, take_n_popular=5000)[0]
data_test_cold = data_test_cold[data_test_cold.item_id != 999999]
data_test_cold = data_test_cold[data_test_cold.user_id.isin(uid)]
data_test_cold = data_test_cold[data_test_cold.user_id.isin(out_users_all)]
test_user_features = user_features[user_features["user_id"].isin(data_test_cold.user_id.unique().tolist())]
print(data_test_cold.user_id.nunique(), test_user_features.user_id.nunique())

items_sold_test = data_test_cold['item_id'].unique().tolist()
items_features_sold_test = item_features[item_features["item_id"].isin(items_sold_test)]

user_features_ext_test = get_users_features(test_user_features, data_test_cold, users_embeddings)[1]
item_features_ext_test= get_items_features(items_features_sold_test, data_test_cold, items_embeddings)[1]

test_cold = test_user_features.user_id.unique().tolist()
out_cold = list(set(out_users_all) - set(test_cold))

Decreased # items from 63056 to 5000
(5000, 7)
801
42 42


In [83]:
# # Отмасштабируем признаки
# from sklearn.preprocessing import StandardScaler
# scale1 = StandardScaler()
# scale2 = StandardScaler()
# user_features_ext = scale1.fit_transform(user_features_ext)
# item_features_ext = scale2.fit_transform(item_features_ext)
# user_features_ext_test = scale1.transform(user_features_ext_test)

In [85]:
#  Построим холодные рекомендации
cold_rec = ColdRecommender(data_train_cold, overall_top_purchases, user_features_ext, item_features_ext)

In [86]:
userid_to_id = cold_rec.userid_to_id

In [87]:
result_cold = data_train_cold.groupby('user_id')['item_id'].unique().reset_index()
result_cold.columns=['user_id', 'actual']

In [99]:
# Строим рекомендации по похожести пользоваелей
result_cold['cold_recommendations'] = result_cold['user_id'].progress_apply(lambda x: \
                                                                             cold_rec.get_similar_users_recommendation(0, user_features_ext.iloc[[userid_to_id[x]]], item_features_ext, N=3))

100%|██████████| 801/801 [00:04<00:00, 198.43it/s]


In [ ]:
# Строим рекомендации по похожести пользоваелей для стандартизованных признаков
# result_cold['cold_recommendations'] = result_cold['user_id'].progress_apply(lambda x: \
#                                                                              cold_rec.get_similar_users_recommendation(x, user_features_ext[userid_to_id[x]], item_features_ext, N=5))

In [103]:
# Рекомендует почему-то одно и тоже, ну возможно данных маловато
result_cold

,user_id,actual,cold_recommendations,cold_top_recommendations
0,1,"[840361, 845307, 852014, 856942, 912676, 94094...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
1,7,"[865569, 886703, 889731, 893400, 1022003, 1029...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
2,8,"[824555, 835576, 841220, 847982, 849843, 85679...","[844839, 873627, 834842]","[844839, 873627, 834842, 1082185, 1029743]"
3,13,"[1106523, 5569230, 840361, 892728, 893802, 904...","[844839, 873627, 834833]","[844839, 873627, 834833, 1082185, 1029743]"
4,16,"[923149, 1035843, 12263788]","[9553397, 6034577, 10284966]","[9553397, 6034577, 10284966, 1082185, 1029743]"
...,...,...,...,...
796,2494,"[849843, 861272, 890739, 910291, 923149, 94252...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
797,2496,"[840361, 871756, 886703, 899624, 916122, 95660...","[844839, 873627, 834842]","[844839, 873627, 834842, 1082185, 1029743]"
798,2497,"[1037840, 1052294, 5569230, 8090537, 1022428, ...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
799,2498,"[824555, 835576, 901776, 904023, 911215, 91749...","[9553397, 6034577, 10284966]","[9553397, 6034577, 10284966, 1082185, 1029743]"


# Train precision

In [77]:
from lightfm.evaluation import precision_at_k, recall_at_k

In [104]:
train_precision = precision_at_k(cold_rec.model, csr_matrix(cold_rec.user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_features_ext).tocsr(),
                                 item_features=csr_matrix(item_features_ext).tocsr(),
                                 k=5).mean()

train_precision

0.49854347

# Test precision

In [94]:

test_user_item_matrix = pd.pivot_table(data_test_cold, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit
sparse_test_user_item = csr_matrix(test_user_item_matrix).tocsr()
test_user_item_matrix.head(2)

item_id,819255,819304,819840,820122,820165,820321,820361,821219,821324,821695,...,15926886,15926927,16053142,16053242,16729299,16769555,16809471,16809649,17104444,17105058
user_id,,,,,,,,,,,,,,,,,,,,,
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [95]:
test_precision = precision_at_k(cold_rec.model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_features_ext_test).tocsr(),
                                 item_features=csr_matrix(item_features_ext_test).tocsr(),
                                 k=5).mean()

test_precision

0.06190476

# Рекомендации, дополненные товарам из топа популярных новым пользователям

In [96]:
users_cold = data[data.user_id.isin(out_users_all)]
users_cold = users_cold.groupby('user_id')['item_id'].unique().reset_index()
users_cold.columns=['user_id', 'actual']

In [101]:
result_cold['cold_top_recommendations'] = result_cold['cold_recommendations'].apply(lambda x: recommender._extend_with_top_popular(list(x), N=5))

In [102]:
result_cold

,user_id,actual,cold_recommendations,cold_top_recommendations
0,1,"[840361, 845307, 852014, 856942, 912676, 94094...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
1,7,"[865569, 886703, 889731, 893400, 1022003, 1029...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
2,8,"[824555, 835576, 841220, 847982, 849843, 85679...","[844839, 873627, 834842]","[844839, 873627, 834842, 1082185, 1029743]"
3,13,"[1106523, 5569230, 840361, 892728, 893802, 904...","[844839, 873627, 834833]","[844839, 873627, 834833, 1082185, 1029743]"
4,16,"[923149, 1035843, 12263788]","[9553397, 6034577, 10284966]","[9553397, 6034577, 10284966, 1082185, 1029743]"
...,...,...,...,...
796,2494,"[849843, 861272, 890739, 910291, 923149, 94252...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
797,2496,"[840361, 871756, 886703, 899624, 916122, 95660...","[844839, 873627, 834842]","[844839, 873627, 834842, 1082185, 1029743]"
798,2497,"[1037840, 1052294, 5569230, 8090537, 1022428, ...","[9553382, 6034577, 10284966]","[9553382, 6034577, 10284966, 1082185, 1029743]"
799,2498,"[824555, 835576, 901776, 904023, 911215, 91749...","[9553397, 6034577, 10284966]","[9553397, 6034577, 10284966, 1082185, 1029743]"
